In [422]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing as ps
import numpy as np
import datetime

Lo primero que haremos será enfocarnos en la codificación de los datos

In [423]:
df = pd.read_csv('../BD_Limpia.csv')

C:\Users\andre\AppData\Local\Temp\ipykernel_17656\3697060408.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../BD_Limpia.csv')


Para ello primero vamos a ver cuales columnas hay que codificar (Aquellas que no sean numericas)
Para esto tomaremos las que son tipo "object" ya que estas pueden ser strings, datetime, etc, el resto podrán ser flotantes, booleanos u otro tipo de valor que no es necesario codificar

In [424]:
df2 = df.copy()
j = 0
for i in [i for i in df.columns.tolist()]:
    if df[i].dtype == object:
        print(i)
        j+=1
print(j)

DESCRIPCION
Mes
Tipo Egreso (INTERNACION)
Paciente Modalidad Contrato
Paciente Regimen Afiliacion
Urg Fecha Ingreso
Urg Fecha Triaje
Urg Fecha Consulta F3
Profesional Especialidad
Dx Principal Tipo 
Dx Principal Cod
Dx Principal Cod.1
Dx Relacionado1 Cod
Dx Relacionado2 Cod 
Dx Relacionado3 Cod
Unidad
Grupo Edad
Paciente Sexo
Clasificacion 1 Triaje 
Enfermeria 
Ubicacion  Urgencias
Causa Externa  (Riesgo)
Modalidad Atencion
Ubicación Triaje
Tiempo Duracion Diligenciamiento F3
Triaje Re-clasificado
Grupo Poblacional
Pertenencia Etnica
ALTO COSTO
28


Ahora tomaremos uno por uno cada valor entregado aqui y se revisará su posible codificación

In [425]:
df['DESCRIPCION'].unique()

array(['URGENCIAS ADULTO', 'URGENCIAS PEDIATRICAS'], dtype=object)

In [426]:
df2['DESCRIPCION'].replace({'URGENCIAS ADULTO': 1, 'URGENCIAS PEDIATRICAS': 0}, inplace=True)

C:\Users\andre\AppData\Local\Temp\ipykernel_17656\1354844995.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df2['DESCRIPCION'].replace({'URGENCIAS ADULTO': 1, 'URGENCIAS PEDIATRICAS': 0}, inplace=True)
C:\Users\andre\AppData\Local\Temp\ipykernel_17656\1354844995.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_dow

In [427]:
df['Tipo Egreso (INTERNACION)'].unique()

array(['No hospitalizacion', 'Hospitalizacion'], dtype=object)

In [428]:
df2['Tipo Egreso (INTERNACION)'].replace({'No hospitalizacion': 1, 'Hospitalizacion': 0}, inplace=True)

C:\Users\andre\AppData\Local\Temp\ipykernel_17656\673998892.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df2['Tipo Egreso (INTERNACION)'].replace({'No hospitalizacion': 1, 'Hospitalizacion': 0}, inplace=True)
C:\Users\andre\AppData\Local\Temp\ipykernel_17656\673998892.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_si

In [429]:
df["Modalidad Atencion"].unique() # Remitido y remitidos son lo mismo

array(['URGENCIAS', 'REMITIDO', 'REMITIDOS'], dtype=object)

In [430]:
df2["Modalidad Atencion"].replace({'URGENCIAS': 1, 'REMITIDO': 0, 'REMITIDOS': 0}, inplace=True)

C:\Users\andre\AppData\Local\Temp\ipykernel_17656\3878965590.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df2["Modalidad Atencion"].replace({'URGENCIAS': 1, 'REMITIDO': 0, 'REMITIDOS': 0}, inplace=True)
C:\Users\andre\AppData\Local\Temp\ipykernel_17656\3878965590.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_

In [431]:
len(df["Dx Principal Cod"].unique()) 
# Al ser tan grande, se codificará con un valor diferente para cada único, 
# pues algunos podrian presentarse mas o tener mas peso

2434

In [432]:
encoder=ps.LabelEncoder()
df2["Dx Principal Cod"]=encoder.fit_transform(df2["Dx Principal Cod"])

Ahora esto nos ha entregado las 2400 enfermedades en numeros en vez de códigos, se evaluará posteriormente la viabilidad de hacerlo de esta manera, ya que supone un peso importante en la matriz de correlaciones.

Para los diagnósticos secundarios se pondrá un 1 en caso de existir uno, y un 0 en caso de que no, se sumarán posteriormente en una única variable.

In [433]:
varabiesl = ['Dx Relacionado3 Cod', 'Dx Relacionado2 Cod ', 'Dx Relacionado1 Cod']

def ifnostringis0(aver):
    if isinstance(aver,str):
        return 1
    else:
        return 0
df2[varabiesl] = df2[varabiesl].applymap(ifnostringis0)
df2['Dx Relacionado3 Cod'].unique()
df2['DxRelacionados'] = df2['Dx Relacionado3 Cod'] + df2['Dx Relacionado1 Cod'] + df2['Dx Relacionado2 Cod ']
df2 = df2.drop(columns=['Dx Relacionado3 Cod', 'Dx Relacionado1 Cod', 'Dx Relacionado2 Cod ', 'Dx Principal Cod.1', "Profesional Especialidad"])
df2['DxRelacionados'].unique()

C:\Users\andre\AppData\Local\Temp\ipykernel_17656\2403177839.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df2[varabiesl] = df2[varabiesl].applymap(ifnostringis0)


array([0, 1, 2, 3], dtype=int64)

In [434]:
df["Mes"].unique()

array(['ENERO', 'FEBRERO', 'MARZO', 'ABRIL', 'MAYO', 'JUNIO', 'JULIO',
       'AGOSTO', 'SEPTIEMBRE', 'OCTUBRE', 'NOVIEMBRE', 'DICIEMBRE'],
      dtype=object)

In [435]:
meses = ['ENERO', 'FEBRERO', 'MARZO', 'ABRIL', 'MAYO', 'JUNIO', 'JULIO','AGOSTO', 'SEPTIEMBRE', 'OCTUBRE', 'NOVIEMBRE', 'DICIEMBRE']
codificacion = {j:i+1 for i,j in enumerate(meses)}
df2["Mes"].replace(codificacion, inplace = True)

C:\Users\andre\AppData\Local\Temp\ipykernel_17656\399413001.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df2["Mes"].replace(codificacion, inplace = True)
C:\Users\andre\AppData\Local\Temp\ipykernel_17656\399413001.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df2["Mes"].replace(codifica

In [436]:
confecha = ["Urg Fecha Ingreso", 'Urg Fecha Triaje', 'Urg Fecha Consulta F3']
def todoASegundos(datetaim):
    try:
        datetaim = datetaim[11:19].split(':')
        segundos = int(datetaim[0])*3600+int(datetaim[1])*60+int(datetaim[2])
        return segundos
    except TypeError:
        return 0

df2[confecha] = df2[confecha].applymap(todoASegundos)
df2[confecha]

C:\Users\andre\AppData\Local\Temp\ipykernel_17656\2211830499.py:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df2[confecha] = df2[confecha].applymap(todoASegundos)


,Urg Fecha Ingreso,Urg Fecha Triaje,Urg Fecha Consulta F3
0,52589,51814,55153
1,66727,66281,71011
2,40756,0,61549
3,38449,37045,43540
4,72274,72038,73021
...,...,...,...
36663,0,10841,13016
36664,0,7847,12244
36665,0,33812,35849
36666,0,40942,43845


In [437]:
df["Dx Principal Tipo "].unique()

array(['NUEVO', 'REPETIDO'], dtype=object)

In [438]:
df2["Dx Principal Tipo "].replace({'NUEVO': 1, 'REPETIDO': 0}, inplace=True)

C:\Users\andre\AppData\Local\Temp\ipykernel_17656\1481903230.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df2["Dx Principal Tipo "].replace({'NUEVO': 1, 'REPETIDO': 0}, inplace=True)
C:\Users\andre\AppData\Local\Temp\ipykernel_17656\1481903230.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

In [439]:
df["ALTO COSTO"].unique() # PARA ESTE SE PONDRÁ 1 SI ES STRING Y 0 SI NO, 
#YA QUE SI ES STRING ES UNA ENFERMEDADE DE ALTO COSTO
df2["ALTO COSTO"] = df2["ALTO COSTO"].apply(ifnostringis0)

In [440]:
df["Paciente Sexo"].unique()

array(['FEMENINO', 'MASCULINO'], dtype=object)

In [441]:
df2["Paciente Sexo"].replace({'FEMENINO': 0, 'MASCULINO': 1}, inplace=True)

C:\Users\andre\AppData\Local\Temp\ipykernel_17656\928752363.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df2["Paciente Sexo"].replace({'FEMENINO': 0, 'MASCULINO': 1}, inplace=True)
C:\Users\andre\AppData\Local\Temp\ipykernel_17656\928752363.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  

Ahora se ejecutarán todas las que se hagan con one-hot-encoding

In [442]:
df2 = pd.get_dummies(df2, columns=["Paciente Modalidad Contrato", "Paciente Regimen Afiliacion", "Grupo Edad"])
df2

,DESCRIPCION,Mes,Tipo Egreso (INTERNACION),Paciente Entidad Responsable Pago,Urg Fecha Ingreso,Urg Fecha Triaje,Urg Fecha Consulta F3,Minutos (Camara - F3),Urg Demora1 Consulta (Minutos),Urg Demora Triaje (Minutos)\n,...,Grupo Edad_5-9,Grupo Edad_50-54,Grupo Edad_55-59,Grupo Edad_60-64,Grupo Edad_65-69,Grupo Edad_70-74,Grupo Edad_75-79,Grupo Edad_80-84,Grupo Edad_85-89,Grupo Edad_> 90
0,1,1,1,5,52589,51814,55153,65.133333,35.133333,9.466667,...,False,False,False,True,False,False,False,False,False,False
1,1,1,1,65,66727,66281,71011,82.050000,52.050000,3.216667,...,False,False,False,True,False,False,False,False,False,False
2,1,1,0,48,40756,0,61549,0.000000,0.000000,0.000000,...,False,False,False,False,False,False,True,False,False,False
3,1,1,0,19,38449,37045,43540,116.850000,86.850000,8.600000,...,False,False,False,False,False,True,False,False,False,False
4,0,1,1,48,72274,72038,73021,21.450000,21.450000,5.083333,...,True,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36663,1,12,1,65,0,10841,13016,50.133333,20.133333,13.883333,...,False,False,False,False,False,False,False,False,False,False
36664,1,12,0,4,0,7847,12244,0.000000,0.000000,0.000000,...,False,False,False,False,False,False,False,False,True,False
36665,1,12,1,18,0,33812,35849,40.900000,10.900000,6.950000,...,False,True,False,False,False,False,False,False,False,False
36666,1,12,1,18,0,40942,43845,0.000000,0.000000,0.000000,...,False,True,False,False,False,False,False,False,False,False


In [443]:
j = 0
for i in [i for i in df2.columns.tolist()]:
    if df2[i].dtype == object:
        print(i)
        j+=1
print(j)

Unidad
Clasificacion 1 Triaje 
Enfermeria 
Ubicacion  Urgencias
Causa Externa  (Riesgo)
Ubicación Triaje
Tiempo Duracion Diligenciamiento F3
Triaje Re-clasificado
Grupo Poblacional
Pertenencia Etnica
9
